In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
df = pd.read_excel('1-s2.0-S2352340918309600-mmc2.xlsx')
df

,Jobs_per_ 1Minute,Jobs_per_ 5 Minutes,Jobs_per_ 15Minutes,Mem capacity,Disk_capacity_GB,Num_of_CPU_Cores,CPU_speed_per_Core,Avg_Recieve_Kbps,Avg_Transmit_Kbps,Class_Name
0,3.000,4.200,4.900,12,128.72872,6,1,27.885,3.96400,'Very Low'
1,4.727,5.545,6.364,12,128.72872,6,1,10.935,10.22700,'Very Low'
2,29.727,31.273,31.909,12,128.72872,6,1,27.598,0.06955,'Low'
3,12.200,13.700,14.800,12,128.72872,6,1,27.885,3.96400,'Low'
4,7.528,8.636,9.364,12,128.72872,6,1,24.653,213.72500,'Very Low'
...,...,...,...,...,...,...,...,...,...,...
2445,12.909,7.545,2.636,4,732.95900,4,3,47.510,41.34300,'Very Low'
2446,1.182,1.000,0.091,4,732.95900,4,3,36.793,84.64600,'Very Low'
2447,2.167,0.833,0.083,4,732.95900,4,3,53.364,79.33200,'Very Low'
2448,4.667,4.250,1.417,4,732.95900,4,3,36.793,84.64600,'Very Low'


In [ ]:
# Normalize the data
scaler = MinMaxScaler()
features = ['Jobs_per_ 1Minute', 'Jobs_per_ 5 Minutes', 'Jobs_per_ 15Minutes',
            'Mem capacity', 'Disk_capacity_GB', 'Num_of_CPU_Cores',
            'CPU_speed_per_Core', 'Avg_Recieve_Kbps', 'Avg_Transmit_Kbps']
df[features] = scaler.fit_transform(df[features])


In [ ]:
# Create sequences for LSTM
sequence_length = 10  # Sequence length for LSTM
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df[features].iloc[i:i+sequence_length].values)
    y.append(df['Class_Name'].iloc[i+sequence_length])  # Use the next value as target

X = np.array(X)
y = pd.get_dummies(y).values  # One-hot encode the target classes

In [ ]:
# Split data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:
# Build LSTM model
model = Sequential([
    LSTM(64, input_shape=(sequence_length, len(features)), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(y.shape[1], activation='softmax')  # Output layer for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.8133 - loss: 0.9299 - val_accuracy: 1.0000 - val_loss: 0.0481
Epoch 2/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8931 - loss: 0.3834 - val_accuracy: 1.0000 - val_loss: 0.0106
Epoch 3/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9017 - loss: 0.3499 - val_accuracy: 1.0000 - val_loss: 0.0040
Epoch 4/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9056 - loss: 0.3081 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 5/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8942 - loss: 0.3443 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 6/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9138 - loss: 0.2847 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 7/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8945 - loss: 0.3354 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 8/20
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8996 - loss: 0.3321 - val_accuracy: 1.0000 - v

In [ ]:
# Evaluate the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("Accuracy:", accuracy_score(y_test_classes, y_pred_classes))
print(classification_report(y_test_classes, y_pred_classes))

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
Accuracy: 1.0
              precision    recall  f1-score   support

           3       1.00      1.00      1.00       488

    accuracy                           1.00       488
   macro avg       1.00      1.00      1.00       488
weighted avg       1.00      1.00      1.00       488

